## Seasonal NAO skill comparison ##

In the decadal predictions, we observe a pretty significant drop in skill post-2005 initiailisation. As the dataset is limited to around 50 years and we have only observed one 'forecast bust' like this, we don't know whether Doug was just lucky or whether something is actually going wrong in the models in the recent period.

To explore this, we consider the seasonal hindcasts:

1. CSF-20C (Coupled Seasonal Forecast):This is a hindcast performed with the model fully coupled to the Nucleus for European Modelling of the Ocean (NEMO) ocean model, initialised from CERA-20C reanalysis data.
2. ASF-20C (Atmospheric Seasonal Forecasts): This is a hindcast performed with prescribed Sea Surface Temperature (SST) and sea-ice boundary conditions at the surface, initialised form ERA-20C reanalysis data.

These both run from initialisation in 1901-2009, so give a longer time period.

Strommen et al. (2023) propose that seasonal forecasts can be used to diagnose decadal forecast signals over a longer period. We plan to look at this in the context of 8 year running mean (years 2-9 of the decadal forecast) NAO predictability. In doing this, we want to explore the following questions:

* Was the recent drop in NAO skill an outlier in the longer period (1901-2020)?
* Have similar 'forecast bust' periods been observed before?
* Are there similar conditions occuring during previous bust periods?
    * i.e. divergence of model/obs SPNA SSTs

In [1]:
# Import relevant modules
import os
import sys

# Third party imports
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

# Import cdo for regridding
from cdo import Cdo
cdo = Cdo()

In [2]:
# Import local dictionaries and functions
sys.path.append('/home/users/benhutch/skill-maps')

# Import dictionaries
import dictionaries as dic

# Set up the path where the functions are stored
sys.path.append('/home/users/benhutch/skill-maps/python')

# Import functions
import functions as func

# Import the NAO skill functions
import nao_skill_functions as nao_func

# Set up the path where the functions are stored
sys.path.append('/home/users/benhutch/skill-maps/rose-suite-matching')

# Import the suite functions
import nao_matching_seasons as nms_func

# Import the bootstrapping functions
import process_bs_values as pbs_func

### Data access issues ###

* ERA20C reanalysis not easily accessible?
* Who should I contact to try to get access?
* Maybe just look back to 1940 onwards (ERA5 window) first

In [3]:
# Find some of the ASF20C data on jasmin
# Set up the parameters
model = "ASF20C" # Alternative is "CSF20C"
variable = "SLP" # Sea level pressure
initialisation = "Nov" # Month of initialisation
year = 1901 # First year of the hindcast
member=1 # Ensemble member
region="Global" # For regridding

# Set up the base path
base_path_20c = "/badc/deposited2020/seasonal-forecasts-20thc/data"

# Find the specific file
folder_name = f"{variable}monthly_{model}_{initialisation}START_ENSmems"

# Set up the file name
file_name = f"{variable}monthly_{year}_M{member}.nc"

# Set up the full path
file_path = os.path.join(base_path_20c, model,
                            folder_name, file_name)

# Check if the file exists
if os.path.exists(file_path):
    print(f"The file {file_path} exists")

# Set up directories in canari to store the regridded psl data
# Set up the base path
base_path = "/gws/nopw/j04/canari/users/benhutch"

# Form a new path
new_path_ASF20C = os.path.join(base_path, "seasonal",
                               "ASF20C", variable,
                                 f"{initialisation}_START")

# Same for the CSF20C
new_path_CSF20C = os.path.join(base_path, "seasonal",
                               "CSF20C", variable,
                                 f"{initialisation}_START")

# Check if the directories exist
if not os.path.exists(new_path_ASF20C):
    print(f"The directory {new_path_ASF20C} exists")
    # Create a new directory
    os.makedirs(new_path_ASF20C, exist_ok=False)

# Same for the CSF20C
if not os.path.exists(new_path_CSF20C):
    print(f"The directory {new_path_CSF20C} exists")
    # Create a new directory
    os.makedirs(new_path_CSF20C, exist_ok=False)

The file /badc/deposited2020/seasonal-forecasts-20thc/data/ASF20C/SLPmonthly_ASF20C_NovSTART_ENSmems/SLPmonthly_1901_M1.nc exists


In [4]:
# Find the gridspec for the file
gridspec_dir = "/home/users/benhutch/gridspec/"

# # Print all of the files within the directory
# print(os.listdir(gridspec_dir))

# Find gridspec-global.txt
gridspec_file = os.path.join(gridspec_dir, "gridspec-global.txt")

# Print the contents of the file
print(open(gridspec_file).read())

gridtype=lonlat
xfirst=-180
xinc=2.5
xsize=144
yfirst=-90
yinc=2.5
ysize=72


In [5]:
# Find the directory where all of the data to be regridded is stored
dir = os.path.join(base_path_20c, model,
                            folder_name)

# Print all of the files within the directory
print(os.listdir(dir))

# Find the files within the directory
files = os.listdir(dir)

# Limit files to the first 10
files = files[:10]

# Loop through the files
for file in files:
    # Set up the output file name
    output_file = os.path.join(new_path_ASF20C, f"{file[:-3]}_rg.nc")

    # If the output file already exists, skip
    if os.path.exists(output_file):
        print(f"The file {output_file} exists")
        continue

    # Set up the input file name
    input_file = os.path.join(dir, file)

    # perform the regridding
    cdo.remapbil(gridspec_file, input=input_file,
                 output=output_file)

['SLPmonthly_1901_M0.nc', 'SLPmonthly_1901_M1.nc', 'SLPmonthly_1901_M10.nc', 'SLPmonthly_1901_M11.nc', 'SLPmonthly_1901_M12.nc', 'SLPmonthly_1901_M13.nc', 'SLPmonthly_1901_M14.nc', 'SLPmonthly_1901_M15.nc', 'SLPmonthly_1901_M16.nc', 'SLPmonthly_1901_M17.nc', 'SLPmonthly_1901_M18.nc', 'SLPmonthly_1901_M19.nc', 'SLPmonthly_1901_M2.nc', 'SLPmonthly_1901_M20.nc', 'SLPmonthly_1901_M21.nc', 'SLPmonthly_1901_M22.nc', 'SLPmonthly_1901_M23.nc', 'SLPmonthly_1901_M24.nc', 'SLPmonthly_1901_M25.nc', 'SLPmonthly_1901_M26.nc', 'SLPmonthly_1901_M27.nc', 'SLPmonthly_1901_M28.nc', 'SLPmonthly_1901_M29.nc', 'SLPmonthly_1901_M3.nc', 'SLPmonthly_1901_M30.nc', 'SLPmonthly_1901_M31.nc', 'SLPmonthly_1901_M32.nc', 'SLPmonthly_1901_M33.nc', 'SLPmonthly_1901_M34.nc', 'SLPmonthly_1901_M35.nc', 'SLPmonthly_1901_M36.nc', 'SLPmonthly_1901_M37.nc', 'SLPmonthly_1901_M38.nc', 'SLPmonthly_1901_M39.nc', 'SLPmonthly_1901_M4.nc', 'SLPmonthly_1901_M40.nc', 'SLPmonthly_1901_M41.nc', 'SLPmonthly_1901_M42.nc', 'SLPmonthly_1901